In [1]:
# Import dependencies
import pandas as pd

# Save csv file name to 'file'
file = "GWDI_orders_coding_copy.csv"

# Import csv file as df
gwdi_df = pd.read_csv(file)

gwdi = gwdi_df.drop(['Name'], axis = 1)

# Preview df
gwdi.head()

,Order #,Order Date,Number Strains Per Order,Strains Ordered,Single/ Multi,Plated,Growing?,Form Spores?,Replated,Shipped,Harvest Plate,Stored,Tested,Unnamed: 14
0,1,4/25/18,1,GWDI_50_D_4,single,4/25/18,yes,NaN,5/4/18,5/7/18,5/8/18,5/10/18,5/14/18,NaN
1,1,4/25/18,2,GWDI_215_G_9,single,4/25/18,not growing,not growing,not growing,not growing,not growing,not growing,not growing,NaN
2,1,4/25/18,3,GWDI_165_A_8,single,4/25/18,yes,yes,5/4/18,5/7/18,5/8/18,5/10/18,5/14/18,NaN
3,1,4/25/18,4,GWDI_26_B_11,single,4/25/18,yes,NaN,5/4/18,5/7/18,5/8/18,5/10/18,5/14/18,NaN
4,2,4/25/18,1,GWDI_295_B_2,multi 14,4/27/18,yes,na,5/9/18,5/14/18,5/11/18,5/16/18,5/21/18,NaN


In [3]:
# Get count of number of GWDI strains ordered
gwdi_df_info = gwdi.describe()

# Get specific count of number of strains ordered, not growing, regular strains
print(f"Of the {gwdi_df_info['Order #'][0]} GWDI wells ordered,")
print(f"here is the break down of those ordered:")
print(f"Wells that have grown: {gwdi['Growing?'].value_counts()[0]}")
print(f"Wells ordered previously: {gwdi['Growing?'].value_counts()[1]}")
print(f"Wells that have not grown: {gwdi['Growing?'].value_counts()[2]}")
print(f"Regular strains ordered: {gwdi['Growing?'].value_counts()[3]}")

Of the 348.0 GWDI wells ordered,
here is the break down of those ordered:
Wells that have grown: 295
Wells ordered previously: 16
Wells that have not grown: 12
Regular strains ordered: 2


In [4]:
# Create df that will contain segments of well coordinates
# Seperate GWDI, well number, row letter, and column number
split_coordinates = gwdi['Strains Ordered'].str.split('_').apply(pd.Series)

# Add plate number of well coordinates to seperate plate number
gwdi['Plate Number'] = split_coordinates[1]

# Print rows where the strain was recorded as 'not growing'
not_growing = gwdi[gwdi['Growing?'].isin(['not growing'])]

# Clean up df and drop unecessary columns
not_growing_clean = not_growing.drop(['Unnamed: 14', 'Form Spores?', 'Replated', 
                                     'Shipped', 'Harvest Plate', 'Stored',
                                     'Tested'], axis = 1)

# Print cleaned up df of non growing strains
not_growing_clean

,Order #,Order Date,Number Strains Per Order,Strains Ordered,Single/ Multi,Plated,Growing?,Plate Number
1,1,4/25/18,2,GWDI_215_G_9,single,4/25/18,not growing,215
98,20,8/1/18,8,GWDI_62_G_11,multi 3,8/24/18,not growing,62
134,22,8/31/18,2,GWDI_26_A_5,single,10/3/18,not growing,26
152,24,10/11/18,6,GWDI_492_B_12,multi 2,10/22/18,not growing,492
198,30,1/15/19,5,GWDI_449_H_1,single,1/29/19,not growing,449
217,33,2/1/19,5,GWDI_35_G_5,multi 2,2/22/19,not growing,35
226,34,2/18/19,3,GWDI_33_E_4,single,3/4/19,not growing,33
278,43,6/10/19,2,GWDI_15_G_12,single,6/10/19,not growing,15
288,46,8/12/19,1,GWDI_26_G_1,single,8/13/19,not growing,26
306,52,9/26/19,3,GWDI_200_A_11,single,9/26/19,not growing,200


In [5]:
# Create list of plate numbers that have had wells not grow
initial_plate_numbers = not_growing['Plate Number'].tolist()

# Get rid of duplicate plates numbers in plate_number_list
# Create empty list
plate_number_list = []

# Loop over initial_plate_numbers and only add number to plate_number_list
# if that number does not already exist
for number in initial_plate_numbers:
    if not number in plate_number_list:
        plate_number_list.append(number)

# Check updated plate number list
plate_number_list

['215', '62', '26', '492', '449', '35', '33', '15', '200', '445']

In [8]:
# Create framework for new df that will contain well orders from plates that have had
# wells not grow
dead_plates = pd.DataFrame({'Order #': [], 'Order Date': [], 'Plated': [],
                           'Strains Ordered': [], 'Growing?': [], 'Plate Number': []})

# Iterrate over gwdi_df and update dead_plates with orders with matching plate numbers 
# to plate_number_list of existing plates with wells that did not grow
for index, row in gwdi.iterrows():
    for plate_number in plate_number_list:
        if row['Plate Number'] == plate_number:
            dead_plates = dead_plates.append({'Order #': row[0],'Order Date': row[1],  'Plated': row[5],
                           'Strains Ordered': row[3], 'Growing?': row[6], 'Plate Number': row[14]}, ignore_index=True)
            
# Preview updated df of dead_plates
dead_plates

,Order #,Order Date,Plated,Strains Ordered,Growing?,Plate Number
0,1.0,4/25/18,4/25/18,GWDI_215_G_9,not growing,215
1,1.0,4/25/18,4/25/18,GWDI_26_B_11,yes,26
2,2.0,4/25/18,4/25/19,GWDI_26_C_5,yes,26
3,11.0,6/14/18,6/15/18,GWDI_445_D_8,yes,445
4,14.0,6/19/18,6/29/18,GWDI_449_F_5,yes,449
5,17.0,7/5/18,7/24/18,GWDI_15_G_1,yes,15
6,20.0,8/1/18,8/24/18,GWDI_35_B_5,yes,35
7,20.0,8/1/18,8/24/18,GWDI_62_G_11,not growing,62
8,21.0,8/28/18,9/25/18,GWDI_200_H_10,yes,200
9,22.0,8/31/18,10/3/18,GWDI_26_A_5,not growing,26
